# Laser pulse propagation in optical fiber

This notebook shows a simple, scalar implementation of the finite difference <a href="#1">[1]</a> for solving the Nonlinear Schrödinger Equation <a href="#2">[2]</a>.

$\frac{\partial A}{\partial z}=-\frac{\alpha}{2}A+i \frac{\beta_2}{2} \frac{\partial^2 A}{\partial t^2}-i \gamma(|A|^2A)$

This nonlinear partial differential equation models how the envelope and phase of light pulse changes when propagating through a single mode optical fiber, when taking power attenuation ($\alpha$), group velocity dispersion ($\beta_2$)and waveguide nonlinearity ($\gamma$) causing self-phase modulation (SPM) into account. A is the slowly varying amplitude of the pulse envelope and t is measured in a frame of reference moving with the pulse at the group velocity $v_g$. The nonlinear Schrödinger equation (for optics) can be derived from the wave equation. However we can choose between two (Fourier transform) notations. The derivation with the negative one can be found in Ursula Keller's book <a href="#3">[3]</a>. I used this, because the scipy library uses the negative convention for the DFT. Depending on the initial width $T_0$ and the peak power $P_0$ of the incident pulse, either dispersive or non linear
effects may dominate along the fiber. It is useful to introduce two length scales, known as the dispersion length $L_D$ and the nonlinear length $L_{NL}$. Let us consider a time scale normalized to the input width $T_0$ as:

$\tau=\frac{t}{T_0}$

In addition, we introduce a normalized amplitude U as:

$A(z,\tau)=\sqrt{P_0}e^{\frac{- \alpha Z}{2}}U(z,\tau)$

We now take into consideration a space scale normalized to the fiber length as:

$\zeta=\frac{z}{L}$

Where L is the fiber length. Thus, it turns out that $U(\zeta,\tau)$ satisfies

$\frac{\partial U}{\partial \zeta}=+i \frac{L}{2 L_D} sgn(\beta_2) \frac{\partial^2 U}{\partial \tau^2}-i e^{- \alpha Z \zeta} \frac{L}{L_{NL}} (|U|^2 U)$

Where $sgn(\beta_2)=\pm 1$ depending on the sign of the coefficient $\beta_2$ and

$L_D=\frac{T_0 ^ 2}{|\beta_2|}$

$L_{NL}=\frac{1}{\gamma_0 P_0}$

The concept behind this normalization process is to exclude any kind of overflow error that may occur during solving the PDE with finite difference method. The derivation of the dimensionless transformation for the nonlinear Schrödinger equation can be found here <a href="#4">[4]</a>. The plotting functions originally came from here <a href="#5">[5]</a>.

## References

<div id="1">[1] Wikimedia Foundation. (2024, November 28). Finite difference method. Wikipedia. https://en.wikipedia.org/wiki Finite_difference_method</div>
<div id="2">[2] Wikimedia Foundation. (2024, November 21). Nonlinear Schrödinger equation. Wikipedia. https://en.wikipedia.org/wiki/Nonlinear_Schr%C3%B6dinger_equation</div>
<div id="3">[3] Keller, U. (2023). Ultrafast lasers: A comprehensive introduction to fundamental principles with practical applications. Springer International Publishing.</div>
<div id="4">[4] Felice, D. (2016, December 1). A Study of a Nonlinear Schrödinger Equation for Optical Fibers. arxiv. https://arxiv.org/pdf/1612.00358</div>
<div id="5">[5] Krarup, O. (n.d.). OLEKRARUP123/NLSE-vector-solver: Code for modelling the nonlinear Schrödinger equation for optical fiber pulse propagation. GitHub. https://github.com/OleKrarup123/NLSE-vector-solver</div>

## Importing the modules

In [ ]:
from libraries import *
from variables import *
from functions import *
from classes import *

## Initialize class about simulation parameters

In [ ]:
%%script false
sim_config = SIM_config(N,Time_window,duration,wavelength0,theta,chirp)

## Testing the defined functions

In [ ]:
%%script false
# Initialize Gaussian pulse in the time domain
testPulse = GaussianPulseTime(sim_config.t,amplitude,sim_config.duration) / amplitude

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
plt.title("Initial pulse")
ax1.plot(sim_config.t,getPower(testPulse)/np.max(getPower(testPulse)),'g-')
ax2.plot(sim_config.t,abs(testPulse)/np.max(abs(testPulse)),'b-')
ax1.set_xlabel("Time [arbitrary unit]")
ax1.set_ylabel("Power [arbitrary unit]", color='g')
ax2.set_ylabel("Amplitude [arbitrary unit]", color='b')
ax1.set_ylim(0,1)
ax2.set_ylim(0,1)
plt.xlim(-5*sim_config.duration,5*sim_config.duration)
savePlot('initial pulse')
plt.show()

## Testing the defined spectrum functions

In [ ]:
%%script false
# Initialize Gaussian pulse in the frequency domain
testSpectrumTheory = GaussianPulseFrequency(sim_config.f,sim_config.frequency0,amplitude,sim_config.duration)
# Getting spectrum from pulse
testSpectrum=getSpectrumFromPulse(sim_config.t,sim_config.f,testPulse)
# Getting Gaussian spectrum
testSpectrum_2=GaussianSpectrum(sim_config.t,sim_config.f,amplitude,1/(sim_config.duration))
plt.figure()
plt.title("Spectrum of initial pulse")
plt.plot(sim_config.f,getPower(testSpectrumTheory)/np.max(getPower(testSpectrumTheory)),label="Spectrum of the theory")
plt.plot(sim_config.f_rel,getPower(testSpectrum)/np.max(getPower(testSpectrum)),label="Spectrum of testPulse")
plt.plot(sim_config.f_rel,getPower(testSpectrum_2)/np.max(getPower(testSpectrum_2)),label="Spectrum generated with GaussianSpectrum")
plt.axis([-1 + sim_config.frequency0,1 + sim_config.frequency0,0,1])
plt.xlabel("Frequency [arbitrary unit]")
plt.ylabel("Power spectral density [arbitrary unit]")
plt.legend()
savePlot('spectrum of the initial pulse')
plt.show()

## Testing the FWHM function

In [ ]:
%%script false
# Something is odd with the product of the pulse and the spectrum's FWHM
FWHM_frequency=FWHM(sim_config.f,getPower(testSpectrum))
FWHM_time=FWHM(sim_config.t,getPower(testPulse))
product=FWHM_frequency*FWHM_time
print(f"The product of the pulse and the spectrum's FWHM is {product}")
print(f"The dispersion length is {dispersion_length} m")
print(f"The nonlinear length is {nonlinear_length} m")

## Initialize class about fiber

In [ ]:
%%script false
fiber=Fiber_config(nsteps,Length,nonlinear_length,dispersion_length,alpha_dB_per_m)
fiber=Fiber_config(nsteps,Length,nonlinear_length,dispersion_length,alpha_dB_per_km)

# Run Simulation

In [ ]:
%%script false
pulseMatrix, spectrumMatrix = Simulation(fiber,sim_config,testPulse,'EFORK3')
testPulse0=np.copy(pulseMatrix[0,:])
testSpectrum0=np.copy(spectrumMatrix[0,:])
testPulseFinal=np.copy(pulseMatrix[-1,:])
testSpectrumFinal=np.copy(spectrumMatrix[-1,:])

## Initialize functions for plotting the results

In [ ]:
%%script false
FWHM_time_final=FWHM(sim_config.t,getPower(testPulseFinal))
FWHM_frequency_final=FWHM(sim_config.f_rel,getPower(testSpectrumFinal))
print("FWHM_time_final",FWHM_time_final)
print("FWHM_frequency_final",FWHM_frequency_final)
plotFirstAndLastPulse(pulseMatrix,sim_config)
plotPulseMatrix2D(pulseMatrix,fiber,sim_config)
plotFirstAndLastSpectrum(spectrumMatrix,sim_config,FWHM_frequency_final)
plotSpectrumMatrix2D(spectrumMatrix,fiber,sim_config,FWHM_frequency_final)
plotSpectrogram(sim_config, pulseMatrix[0,:], 50, 2000, sim_config.time_step, label='initial')
plotSpectrogram(sim_config, pulseMatrix[-1,:], 50, 2000, sim_config.time_step, label='final')

## Fractional order between 0.1 and 1.0 - The time domain

In [ ]:
%%script false
#EFORK3 N= 2**11; Time_window=2000; nsteps=2**10
alphas = [1.0]
t=sim_config.t
plt.figure()
testPulse = chirpedGaussianPulseTime(sim_config.t,amplitude,sim_config.duration, sim_config.chirp) / amplitude
power = getPower(testPulse)
maximum_power=np.max(power)
plt.plot(t,power/maximum_power,label="Initial pulse")
myfile = open('results.txt', 'w')
for element in alphas:
    sim_config = SIM_config(N,Time_window,frequency0,wavelength0,element, chirp)
    pulseMatrix, spectrumMatrix = Simulation(fiber,sim_config,testPulse,'EFORK3')
    plt.plot(t,getPower(pulseMatrix[-1,:])/maximum_power,label=f"Fractional order: {element}")
    testPulseFinal=np.copy(pulseMatrix[-1,:])
    testSpectrumFinal=np.copy(spectrumMatrix[-1,:])
    
    FWHM_time_final=analyze_pulse_characteristics(sim_config.t, getPower(testPulseFinal), 0.5)
    FWHM_frequency_final=analyze_pulse_characteristics(sim_config.f_rel,getPower(testSpectrumFinal), 0.5)
    FW5percentM_time_final = analyze_pulse_characteristics(sim_config.t, getPower(testPulseFinal), 0.05)
    FW5percentM_frequency_final=analyze_pulse_characteristics(sim_config.f_rel,getPower(testSpectrumFinal), 0.05)
    
    myfile.write(f"{element},{FWHM_time_final},{FWHM_frequency_final},{FW5percentM_time_final},{FW5percentM_frequency_final}\n")
myfile.close()
plt.title("Fractional order between 0.5 and 1.0")
plt.axis([-10,10,0,1])
plt.xlabel("Time [arbitrary unit]")
plt.ylabel("Power [arbitrary unit]")
#savePlot('Fractional order between 0.1 and 1.0')
plt.legend(bbox_to_anchor=(1.05,0.3))
plt.show()

## Fractional order between 0.1 and 1.0 - The frequency domain

In [ ]:
%%script false
#EFORK3 N= 2**11; Time_window=3000; nsteps=2**10
alphas = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
plt.figure()
testPulse = chirpedGaussianPulseTime(sim_config.t,amplitude,sim_config.duration, sim_config.chirp) / amplitude
testSpectrum = getSpectrumFromPulse(sim_config.t,sim_config.f,testPulse)
power = getPower(testSpectrum)
maximum_power=np.max(power)
plt.plot(sim_config.f,power/maximum_power,label="Initial pulse")
for element in alphas:
    sim_config = SIM_config(N,Time_window,frequency0,wavelength0,element, chirp)
    _, spectrumMatrix = Simulation(fiber,sim_config,testPulse,'EFORK3')
    plt.plot(sim_config.f,getPower(spectrumMatrix[-1,:])/maximum_power,label=f"Fractional order: {element}")
plt.title("Fractional order between 0.5 and 1.0")
plt.axis([-0.5,0.5,0,1])
plt.xlabel("Frequency [arbitrary unit]")
plt.ylabel("Spectral Power [arbitrary unit]")
#savePlot('Fractional order between 0.1 and 1.0')
plt.legend(bbox_to_anchor=(1.05,0.3))
plt.show()

## Checking how reducing the number of steps affects pulse accuracy

In [ ]:
%%script false

#First, make a copy of the pulse computed with the original fiber settings and nsteps
baselinePulse = np.copy(testPulseFinal) # reference with 2**14 steps 

#Make array of the number of steps
#nsteps_list=np.logspace(0,np.log10(fiber.nsteps),9)  
nsteps_list=[2**13, 2**14, 2**15]
      
#Pre-define arrays for holding errors
err_list=np.zeros_like(nsteps_list)*1.0

#Start figure and plot the pulse with the large number of steps
plt.figure()
plt.plot(sim_config.t,getPower(baselinePulse)/np.max(getPower(baselinePulse)),label=f"Baseline for nsteps = {fiber.nsteps}")

#For each iteration, define a new fiber with a different number of steps but the same physical properties
for i in range(0,len(nsteps_list)):
  nsteps_i=int(nsteps_list[i])
  fiber_i=Fiber_config(nsteps_i,Length,nonlinear_length,dispersion_length,alpha_dB_per_km)
  
  #Run the simulation and plot the computed pulse
  pulseMatrix_i,_= Simulation(fiber_i,sim_config,testPulse,'EFORK3')
  
  pulse_i=pulseMatrix_i[-1,:]
  
  plt.plot(sim_config.t,getPower(pulse_i)/np.max(getPower(pulse_i)),label=f"Pulse for nsteps = {nsteps_i}")
  
  #Compute and store errors
  err = getEnergy(sim_config.t,(pulse_i-baselinePulse))/getEnergy(sim_config.t, baselinePulse)*100
  err_list[i]=err

plt.xlabel("Time [a.u.]")
plt.ylabel("Power [a.u.]")
plt.title("Impulse solutions for different nsteps with EFORK3 method")
plt.legend(bbox_to_anchor=(1.05,0.3))
plt.show()

## Convergence of the solution

In [ ]:
%%script false

x=np.log10(nsteps_list)
y=np.log10(err_list/np.max(err_list))

model = np.polyfit(x, y, 1)
print("Model parameters [a,b]=",model)
y_model=np.poly1d(model)
plt.figure()
plt.plot(x,y_model(x),'r-',label=f"$err(1 step)*nsteps^{  {np.round(model[0],3)}}$")
plt.plot(x,y,'.')
plt.xlabel('log10(nsteps)')
plt.ylabel('log10(err/err_max)')
plt.title("Converfence of the solution with EFORK3 method")
plt.legend()
plt.show()

## Chirped Gaussian pulse for GVD term only

In [ ]:
%%script false
#EFORK3 N= 2**10; Time_window=1500; nsteps=2**15
chirp_parameters = [-2, 0, 2]
normalized_zlocs_array = np.linspace(0,fiber.length,fiber.nsteps) / fiber.dispersion_length
plt.figure()
for m in chirp_parameters:
    sim_config = SIM_config(N,Time_window,duration,wavelength0,theta,m)
    fiber = Fiber_config(nsteps,Length,nonlinear_length,dispersion_length,alpha_dB_per_km)
    testPulse = chirpedGaussianPulseTime(sim_config.t,amplitude,sim_config.duration,sim_config.chirp) / amplitude
    pulseMatrix, _ = Simulation(fiber,sim_config,testPulse,'EFORK3')
    testPulse0 = np.copy(pulseMatrix[0,:])
    FWHM_testPulse0 = analyze_pulse_characteristics(sim_config.t,getPower(testPulse0)/np.max(getPower(testPulse0)),0.5)
    broadening_factor_simulation = np.zeros(fiber.nsteps)
    for n in range(fiber.nsteps):
        testPulse_index = np.copy(pulseMatrix[n,:])
        FWHM_testPulse_index = analyze_pulse_characteristics(sim_config.t,getPower(testPulse_index)/np.max(getPower(testPulse_index)),0.5)
        broadening_factor_simulation[n] = FWHM_testPulse_index / FWHM_testPulse0
    plt.plot(normalized_zlocs_array,broadening_factor_simulation,'.',label=f"Chirp parameter = {m}")
plt.xlabel('z/L_D')
plt.ylabel('Broadening factor')
plt.title("Broadening factor vs. normalized propagation distance for different chirp parameters")
#plt.axis([0,0.05,0,5])
plt.legend()
plt.show()

## Chirped Gaussian pulse for SPM term only

In [ ]:
%%script false
#EFORK3 N= 2**10; Time_window=5000; nsteps=2**16
chirp_parameters = [-20, -10, 0, 10]
fiber_length = 4.5 * np.pi * nonlinear_length
plt.figure()
for m in chirp_parameters:
    sim_config = SIM_config(N,Time_window,duration,wavelength0,theta,m)
    fiber = Fiber_config(nsteps,fiber_length,nonlinear_length,dispersion_length,0)
    testPulse = chirpedGaussianPulseTime(sim_config.t,amplitude,sim_config.duration,sim_config.chirp) / amplitude
    _, spectrumMatrix = Simulation(fiber,sim_config,testPulse,'Euler')
    power = getPower(spectrumMatrix[-1,:])
    maximum_power=np.max(power)
    plt.plot(sim_config.f,power/maximum_power,'-',label=f"Chirp parameter = {m}")
plt.xlabel("Frequency [arbitrary unit]")
plt.ylabel("Power spectral density [arbitrary unit]")
plt.title("Power spectral density vs. frequency for different chirp parameters")
plt.axis([-2,2,0,1])
plt.legend()
plt.show()

## Initialize classes

In [ ]:
#%%script false
sim_config = SIM_config2(N,Time_window,duration,wavelength0,theta,chirp)
fiber = Fiber_config2(nsteps,Length,nonlinear_length,dispersion_length,alpha_dB_per_m,beta2,gammaconstant)

## Some safety stuff

In [ ]:
#%%script false
# Compute stability step sizes
omega_max = np.max(sim_config.omega)
dz_est = estimate_stable_dz(fiber.gammaconstant, peak_power, wavelength0, n0, fiber.beta2, omega_max)
dt_est = estimate_grid_resolution(duration, omega_max)
print('Stable dt value: ',dt_est)
print('Stable dz value: ',dz_est)

# Calculate critical peak power
P_crit = wavelength0**2 / (2*np.pi*n0*nonlinear_refractive_index)
P_0 = peak_power
if P_0 < P_crit:
    print(f"Crytical power condition is not satisfied for self-focusing!")
else:
    raise ValueError(f'Crytical power condition is satisfied for self focusing!')

# Calculate nonlinear phase shift limit
phi_nl = fiber.gammaconstant*peak_power*fiber.length
if phi_nl < 2*np.pi:
    print(f"Clean pulse propagation or compression is achieved!")
else:
    raise ValueError('Significant SPM and pulse distortion!')
# Intensity and physical effect regimes
if peak_intensity <= 1e15:
    print("Linear propagation regime!")
elif (peak_intensity > 1e15 and  peak_intensity <= 1e17):
    print("Kerr + SPM regime!")
elif (peak_intensity > 1e17 and  peak_intensity <= 3e17):
    raise ValueError('Self-focusing threshold regime!')
elif (peak_intensity >= 1e18):
    raise ValueError('Ionization/damage regime!')

# You want N ≥ 1 for at least a fundamental soliton.
N_soliton = np.sqrt(fiber.gammaconstant * peak_power * duration**2 / np.abs(fiber.beta2))
#print("Soliton number:", N_soliton)

## Fractional order between 0.9 and 1.0 - The time domain with MFEMIP

In [ ]:
%%script false
#Length=1e-4
alphas = [0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0]
plt.figure()
#testPulse = GaussianPulseTime(sim_config.t,amplitude,sim_config.duration)
testPulse = chirpedGaussianPulseTime(sim_config.t,amplitude,sim_config.duration, sim_config.chirp)
power = getPower(testPulse)
maximum_power=np.max(power)
plt.plot(sim_config.t,power/maximum_power,label="Initial pulse")
myfile = open('results.txt', 'w')
for element in alphas:
    sim_config = SIM_config2(N,Time_window,frequency0,wavelength0,element, chirp)
    A_history, A_spectrum_history, PhotonNumber_values = MFEMIP(fiber,sim_config,testPulse)
    maximum_PhotonNumber = np.max(PhotonNumber_values)
    last_normalized_PhotonNumber = PhotonNumber_values[-1] / maximum_PhotonNumber
    plt.plot(sim_config.t,getPower(A_history[-1])/maximum_power,label=f"Fractional order: {element}")
    testPulseFinal=np.copy(A_history[-1])
    testSpectrumFinal=np.copy(A_spectrum_history[-1])
    
    FWHM_time_final=analyze_pulse_characteristics(sim_config.t, getPower(testPulseFinal), 0.5)
    FWHM_omega_final=analyze_pulse_characteristics(sim_config.omega,getPower(testSpectrumFinal), 0.5)
    FW5percentM_time_final = analyze_pulse_characteristics(sim_config.t, getPower(testPulseFinal), 0.05)
    FW5percentM_omega_final=analyze_pulse_characteristics(sim_config.omega,getPower(testSpectrumFinal), 0.05)
    
    myfile.write(f"{element},{FWHM_time_final},{FWHM_omega_final},{FW5percentM_time_final},{FW5percentM_omega_final}, {last_normalized_PhotonNumber}\n")
myfile.close()
plt.title("Fractional order between 0.9 and 1.0")
#plt.axis([-5*sim_config.duration,5*sim_config.duration,0,1])
plt.xlabel("Time [s]")
plt.ylabel("Normalized power [arbitrary unit]")
#plt.legend(bbox_to_anchor=(1.05,0.3))
plt.legend()
plt.show()

## Fractional order between 0.9 and 1.0 - The frequency domain with MFEMIP

In [ ]:
%%script false
#Length=1e-4
alphas = [0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0]
plt.figure()
#testPulse = GaussianPulseTime(sim_config.t,amplitude,sim_config.duration)
testPulse = chirpedGaussianPulseTime(sim_config.t,amplitude,sim_config.duration, sim_config.chirp)
testSpectrum = getSpectrumFromPulse(sim_config.t,testPulse)
power = getPower(testSpectrum)
maximum_power=np.max(power)
plt.plot(sim_config.omega,power/maximum_power,label="Initial pulse")
for element in alphas:
    sim_config = SIM_config2(N,Time_window,frequency0,wavelength0,element, chirp)
    _, A_spectrum_history, _ = MFEMIP(fiber,sim_config,testPulse)
    plt.plot(sim_config.omega,getPower(A_spectrum_history[-1])/maximum_power,label=f"Fractional order: {element}")
plt.title("Fractional order between 0.9 and 1.0")
plt.xlabel("Angular Frequency [rad/s]")
plt.ylabel("Normalized Spectral Power [arbitrary unit]")
#plt.axis([-0.1e16,0.1e16,0,1])
#savePlot('Fractional order between 0.1 and 1.0')
#plt.legend(bbox_to_anchor=(1.05,0.3))
plt.legend()
plt.show()

## Photon number conservation

In [ ]:
%%script false
#Length=1e-4
alphas = [0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0]
plt.figure()
#testPulse = GaussianPulseTime(sim_config.t,amplitude,sim_config.duration)
testPulse = chirpedGaussianPulseTime(sim_config.t,amplitude,sim_config.duration, sim_config.chirp)
for element in alphas:
    sim_config = SIM_config2(N,Time_window,frequency0,wavelength0,element, chirp)
    _, _, PhotonNumber_values = MFEMIP(fiber,sim_config,testPulse)
    plt.plot(fiber.zlocs_array,PhotonNumber_values,label=f"Fractional order: {element}")
    #print((PhotonNumber_values[0]-PhotonNumber_values[-1])/PhotonNumber_values[0])
#savePlot('Fractional order between 0.9 and 1.0')
plt.xlabel('Propagation distance [m]')
plt.ylabel('Photon number [count]')
plt.title('Photon number conservation')
#plt.legend(bbox_to_anchor=(1.05,0.3))
plt.legend()
plt.show()

## Chirped Gaussian pulse for GVD term only with MFEMIP

In [ ]:
%%script false
#Length=1e-4
chirp_parameters = [-2, 0, 2]
normalized_zlocs_array = fiber.zlocs_array / fiber.dispersion_length
plt.figure()
for m in chirp_parameters:
    sim_config = SIM_config2(N,Time_window,duration,wavelength0,theta,m)
    fiber = Fiber_config2(nsteps,Length,nonlinear_length,dispersion_length,0,beta2,0)
    testPulse = chirpedGaussianPulseTime(sim_config.t,amplitude,sim_config.duration,sim_config.chirp)
    A_history, _, _ = MFEMIP(fiber,sim_config,testPulse)
    testPulse0 = np.copy(A_history[0])
    FWHM_testPulse0 = analyze_pulse_characteristics(sim_config.t,getPower(testPulse0)/np.max(getPower(testPulse0)),0.5)
    broadening_factor_simulation = np.zeros(fiber.nsteps)
    for n in range(fiber.nsteps):
        testPulse_index = np.copy(A_history[n])
        FWHM_testPulse_index = analyze_pulse_characteristics(sim_config.t,getPower(testPulse_index)/np.max(getPower(testPulse_index)),0.5)
        broadening_factor_simulation[n] = FWHM_testPulse_index / FWHM_testPulse0
    plt.plot(normalized_zlocs_array,broadening_factor_simulation,'.',label=f"Chirp parameter = {m}")
plt.xlabel('z/L_D')
plt.ylabel('Broadening factor')
plt.title("Broadening factor vs. normalized propagation distance for different chirp parameters")
plt.legend()
plt.show()

## Chirped Gaussian pulse for SPM term only with FL1_direct_NL

In [ ]:
%%script false
#Length=1e-4
chirp_parameters = [10]
fiber_length = 4.5 * np.pi * nonlinear_length
plt.figure()
for m in chirp_parameters:
    sim_config = SIM_config2(N,Time_window,duration,wavelength0,theta,m)
    fiber = Fiber_config2(nsteps,fiber_length,nonlinear_length,dispersion_length,0,0,gammaconstant)
    testPulse = chirpedGaussianPulseTime(sim_config.t,amplitude,sim_config.duration,m)
    SSFM_A_spectrum_history, FEM1_A_spectrum_history, FEM2_A_spectrum_history, FEM3_A_spectrum_history = FL1_direct_NL(fiber,sim_config,testPulse)
    power1 = getPower(FEM1_A_spectrum_history[-1])
    maximum_power1=np.max(power1)
    power2 = getPower(SSFM_A_spectrum_history[-1])
    maximum_power2=np.max(power2)
    power3 = getPower(FEM2_A_spectrum_history[-1])
    maximum_power3=np.max(power3)
    power4 = getPower(FEM3_A_spectrum_history[-1])
    maximum_power4=np.max(power4)
    #plt.plot(sim_config.f / sim_config.f0,power1/maximum_power1,'-',label=f"Chirp parameter = {m}, FEM1")
    plt.plot(sim_config.f / sim_config.f0,power2/maximum_power2,'--',label=f"Chirp parameter = {m}, SSFM")
    #plt.plot(sim_config.f / sim_config.f0,power3/maximum_power3,'-',label=f"Chirp parameter = {m}, FEM2")
    plt.plot(sim_config.f / sim_config.f0,power4/maximum_power4,'-',label=f"Chirp parameter = {m}, MFEM")
plt.xlabel("Normalized Frequency")
plt.ylabel("Normalized Power spectral density [arbitrary unit]")
plt.title("Normalized Power spectral density vs. normalized frequency")
plt.axis([-4,4,0,1])
plt.legend()
plt.show()

## FWHM_time_final

In [ ]:
#%%script false
#Length=1e-4
alphas = [0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0]
FWHM_time_final = [
    1.672139413734577e-14,
    1.5977562737544805e-14,
    1.5345655808921334e-14,
    1.4823502387490166e-14,
    1.4402460404227133e-14,
    1.4070711207550877e-14,
    1.3814651934241598e-14,
    1.3621159755633962e-14,
    1.3478073301501978e-14,
    1.337460897456946e-14,
    1.3302195421229825e-14                 
]
plt.figure()
plt.plot(alphas,FWHM_time_final,'-')
plt.xlabel("Fractional order")
plt.ylabel("FWMH time [s]")
plt.title("Fractional order vs. FWMH time")
plt.legend()
plt.show()

## FWHM_omega_final

In [ ]:
#%%script false
#Length=1e-4
alphas = [0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0]
FWHM_omega_final = [
    282821988999397.1,
    263258699144394.78,
    249271909583720.8,
    242251752754121.88,
    236468013054801.66,
    231736598394579.5,
    227876682168659.56,
    224728065918314.84,
    222155954625816.7,
    220049991269773.7,
    218321108434478.62                   
]
plt.figure()
plt.plot(alphas,FWHM_omega_final,'-')
plt.xlabel("Fractional order")
plt.ylabel("FWMH angular frequency [rad/s]")
plt.title("Fractional order vs. FWMH angular frequency")
plt.legend()
plt.show()

## FW5percentM_time_final

In [ ]:
#%%script false
#Length=1e-4
alphas = [0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0]
FW5percentM_time_final = [
    2.976827247779946e-14,
    2.932670489519572e-14,
    2.894864977255725e-14,
    2.8632046052165943e-14,
    2.8374853282418843e-14,
    2.8169135951669773e-14,
    2.8008372463147516e-14,
    2.7885652985118216e-14,
    2.7794729242321764e-14,
    2.7728432511197117e-14,
    2.7681276711090796e-14            
]
plt.figure()
plt.plot(alphas,FW5percentM_time_final,'-')
plt.xlabel("Fractional order")
plt.ylabel("FW5percentM time [s]")
plt.title("Fractional order vs. FW5percentM time")
plt.legend()
plt.show()

## FW5percentM_omega_final

In [ ]:
#%%script false
#Length=1e-4
alphas = [0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0]
FW5percentM_omega_final = [
    855137542153450.1,
    835335306739719.5,
    810349708101498.0,
    779228325636804.5,
    746413893275102.2,
    720672921501098.0,
    689651930790165.0,
    652586327002105.6,
    616161036722477.8,
    587629313329481.5,
    558612547122156.1       
]
plt.figure()
plt.plot(alphas,FW5percentM_omega_final,'-')
plt.xlabel("Fractional order")
plt.ylabel("FW5percentM angular frequency [rad/s]")
plt.title("Fractional order vs. FW5percentM angular frequency")
plt.legend()
plt.show()

## last_normalized_PhotonNumber

In [ ]:
#%%script false
#Length=1e-4
alphas = [0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0]
last_normalized_PhotonNumber = [
    0.4572001636357399,
    0.5505870835789389,
    0.6395723331990233,
    0.7206578922177476,
    0.7915515245355212,
    0.8512276685298016,
    0.8997401751047962,
    0.9379096456777026,
    0.9669921890574342,
    0.988360650188707,
    1.0            
]
plt.figure()
plt.plot(alphas,last_normalized_PhotonNumber,'-')
plt.xlabel("Fractional order")
plt.ylabel("Last normalized photon number")
plt.title("Fractional order vs. Last normalized photon number")
plt.legend()
plt.show()

## Fractional order between 0.9 and 1.0 - another study in the frequency domain

In [ ]:
%%script false
#Length=1e-4
alphas = [0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0]
fiber_length = 4.5 * np.pi * nonlinear_length
plt.figure()
myfile = open('results2.txt', 'w')
for m in alphas:
    sim_config = SIM_config2(N,Time_window,duration,wavelength0,m,chirp)
    fiber = Fiber_config2(nsteps,fiber_length,nonlinear_length,dispersion_length,0,0,gammaconstant)
    testPulse = chirpedGaussianPulseTime(sim_config.t,amplitude,sim_config.duration,sim_config.chirp)
    _, A_spectrum_history, _ = MFEMIP(fiber,sim_config,testPulse)
    power = getPower(A_spectrum_history[-1])
    maximum_power=np.max(power)
    FWHM_omega_final=analyze_pulse_characteristics(sim_config.omega,power, 0.5)
    FW5percentM_omega_final=analyze_pulse_characteristics(sim_config.omega,power, 0.05)
    myfile.write(f"{m},{FWHM_omega_final},{FW5percentM_omega_final}\n")
    plt.plot(sim_config.f / sim_config.f0,power/maximum_power,'-',label=f"Fractional order: {m}")
myfile.close()
plt.xlabel("Normalized frequency")
plt.ylabel("Normalized Power spectral density [arbitrary unit]")
plt.title("Normalized Power spectral density vs. angular frequency for different fractional order")
plt.axis([-7,7,0,1])
plt.legend()
plt.show()

## Initial spectrum

In [ ]:
testPulse = chirpedGaussianPulseTime(sim_config.t,amplitude,sim_config.duration, sim_config.chirp)
testSpectrum=getSpectrumFromPulse(sim_config.t,testPulse)
plt.figure()
plt.title("Spectrum of initial pulse")
plt.plot(sim_config.omega,getPower(testSpectrum)/np.max(getPower(testSpectrum)),label="Spectrum of testPulse")
plt.xlabel("Angular [rad/s]")
plt.ylabel("Power spectral density [arbitrary unit]")
plt.axis([-0.5e15 ,0.5e15,0,1])
plt.legend()
plt.show()
FWHM_omega_final=analyze_pulse_characteristics(sim_config.omega,getPower(testSpectrum), 0.05)
print(FWHM_omega_final)

## FW5percentM_omega_final

In [ ]:
#%%script false
#Length=1e-4
alphas = [0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0]

FW5percentM_omega_initial = [
    454818859559033.4,
    454818859559033.4,
    454818859559033.4,
    454818859559033.4,
    454818859559033.4,
    454818859559033.4,
    454818859559033.4,
    454818859559033.4,
    454818859559033.4,
    454818859559033.4,
    454818859559033.4,          
]

FW5percentM_omega_final_zero_chirp = [
    855137542153450.1,
    835335306739719.5,
    810349708101498.0,
    779228325636804.5,
    746413893275102.2,
    720672921501098.0,
    689651930790165.0,
    652586327002105.6,
    616161036722477.8,
    587629313329481.5,
    558612547122156.1          
]

FW5percentM_omega_final_minus10_chirp = [
    4958646258839506.0,
    4890836947134092.0,
    4834057070144676.0,
    4777094375113928.0,
    4730537895518934.0,
    4686989511539655.0,
    4646067789054646.0,
    4614462399129263.0,
    4585636726698075.0,
    4559479150278567.0,
    4535919162426514.0         
]

FW5percentM_omega_final_plus10_chirp = [
    4180289042886963.0,
    4119445478022740.5,
    4079190679802465.0,
    4054606065439456.0,
    4043353559664248.0,
    4042636531822714.0,
    4049669371722956.5,
    4061977533127723.0,
    4077537415385393.5,
    4094799053103497.5,
    4112640254196635.0             
]

plt.figure()
plt.plot(alphas,FW5percentM_omega_initial,label="Initial")
plt.plot(alphas,FW5percentM_omega_final_zero_chirp,label="C=0")
plt.plot(alphas,FW5percentM_omega_final_minus10_chirp,label="C=-10")
plt.plot(alphas,FW5percentM_omega_final_plus10_chirp,label="C=+10")
plt.xlabel("Fractional order")
plt.ylabel("FW5percentM angular frequency [rad/s]")
plt.title("Fractional order vs. FW5percentM angular frequency")
plt.legend()
plt.show()

## Fractional order between 0.9 and 1.0 - another study in the frequency domain

In [ ]:
#%%script false
#Length=1e-4
chirp_values = np.arange(-150, 0, 10)
FWHM_omega_final_values = []
FW5percentM_omega_final_values = []
fiber_length = 4.5 * np.pi * nonlinear_length
myfile = open('results3.txt', 'w')
for m in chirp_values:
    sim_config = SIM_config2(N,Time_window,duration,wavelength0,theta,m)
    fiber = Fiber_config2(nsteps,fiber_length,nonlinear_length,dispersion_length,0,0,gammaconstant)
    testPulse = chirpedGaussianPulseTime(sim_config.t,amplitude,sim_config.duration,m)
    _, A_spectrum_history, _ = MFEMIP(fiber,sim_config,testPulse)
    power = getPower(A_spectrum_history[-1])
    maximum_power=np.max(power)
    FWHM_omega_final=analyze_pulse_characteristics(sim_config.omega,power, 0.5)
    FW5percentM_omega_final=analyze_pulse_characteristics(sim_config.omega,power, 0.05)
    FWHM_omega_final_values.append(FWHM_omega_final)
    FW5percentM_omega_final_values.append(FW5percentM_omega_final)
    myfile.write(f"{m},{FWHM_omega_final},{FW5percentM_omega_final}\n")
myfile.close()
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
plt.title(f"Optimal chirp for fractional order: {theta}")
ax1.plot(chirp_values,FW5percentM_omega_final_values,'g-')
ax2.plot(chirp_values,FWHM_omega_final_values,'b-')
ax1.set_xlabel("Chirp parameter")
ax1.set_ylabel("5% power bandwidth [rad/s]", color='g')
ax2.set_ylabel("50% power bandwidth [rad/s]", color='b')
plt.show()